In [ ]:
import os

In [ ]:
os.path.abspath('../')

In [ ]:
os.path.abspath('../') + '/sir_model_v1.json'
path = '/Users/sararozin/Documents/Tama/Tama/' # replace with your path, coppy os.path.abspath('../') output

In [2]:
os.path.abspath('../') + '/sir_model_v1.json'
path = '/Users/sararozin/Documents/Tama/Tama/' # replace with your path, coppy os.path.abspath('../') output

'/Users/sararozin/Documents/Tama/Tama/total_request_per_year/models/sir_model_v1.json'

In [3]:
# Save the model
import json
def save_sir_params(city, beta, gamma):
    print('save_sir_params', city, beta, gamma)
    model = {
        'SemelIishuv': int(city),
        'beta': float(beta),
        'gamma': float(gamma),
    }
    sir_models = [model]
    with open("/Users/sararozin/Documents/Tama/Tama/total_request_per_year/models/sir/sir_model_v1.json", "r") as sir_model_file:
        sir_models = json.load(sir_model_file)
        sir_models.append(model)
    with open("/Users/sararozin/Documents/Tama/Tama/total_request_per_year/models/sir/sir_model_v1.json", 'w') as sir_model_file:
        json.dump(sir_models, sir_model_file,
                  indent=4,
                  separators=(',', ': '))

In [4]:
import pandas as pd
import numpy as np 
from matplotlib import pyplot as plt
from scipy.integrate import odeint
from scipy.optimize import curve_fit, minimize

df = pd.read_csv('1.csv')

df = df[["SemelIishuv", "ShnatBakasha"]]
df = df[pd.to_numeric(df['ShnatBakasha'], errors='coerce').notnull()]

# print(df)

data = {}

for ind in df.index:
    yeshuv = df["SemelIishuv"][ind]
    
    if yeshuv not in data:
        data[yeshuv] = {}
        
        for year in range(2000, 2023):
            data[yeshuv][year] = 0
            
    year = int(df['ShnatBakasha'][ind])
    
    data[yeshuv][year] += 1

def SIR(X, t, beta, gamma, N):
    """
    S = X[0], I=X[1], R=X[2]
    """
    dSdt = -beta*X[0]*X[1]/N
    dIdt = -dSdt - gamma*X[1]
    dRdt = gamma*X[1]
    return [dSdt, dIdt, dRdt]

def infectedSIR(t, beta, gamma):
    # N = 6*10**7
    N = 10000
    I0 = 1
    X0 = [N-I0, I0, 0]
    return odeint(SIR, X0, t, args=(beta, gamma, N))[:,1]

def min_diff(x):
    global x_data, t_data

    guess = infectedSIR(t_data, x[0], x[1])
    guess /= np.sum(guess) 
    guess *= np.sum(x_data) 

    return np.sum(np.square(guess - x_data))

result = []

for yeshuv in data:
    print(yeshuv, data[yeshuv])
    x_data = np.array([data[yeshuv][i] for i in range(2000, 2023)])

    
    # kernel_size = 3
    # kernel = np.ones(kernel_size) / (kernel_size)
    # x_data = np.convolve(x_data, kernel, mode="valid")
    
    x0 = [0.9, 0.2]
    t_data = np.linspace(0, 22, 23)

    # kernel_size = 3
    # kernel = np.ones(kernel_size) / (kernel_size)
    # x_data = np.convolve(x_data, kernel, mode="valid")

    res = minimize(min_diff, x0, method='Nelder-Mead', tol=1e-6)
    x = res.x
    save_sir_params(yeshuv, x[0], x[1])
    t_data = np.linspace(0, 30, 31)

    guess = infectedSIR(t_data, x[0], x[1])
    guess /= np.sum(guess) 
    guess *= np.sum(x_data) 
    
    t = np.array([i - 2000 for i in range(2000, 2023)])
    
    plt.title("City code %d" % yeshuv)
    plt.plot(t_data, guess, label = "Guess till 2030")
    plt.plot(t, x_data, label = "Current data")
    plt.legend()
    plt.savefig("report\\" + str(yeshuv) + ".png", dpi = 120)
    plt.clf()   
    
    result.append([yeshuv] + list(guess[23:31]))
    
    # plt.show()

print(result)

df = pd.DataFrame(result, columns=['Yeshuv'] + [i for i in range(23, 31)])
df.to_csv('out_v2.csv')





# plt.show()

28 {2000: 0, 2001: 0, 2002: 0, 2003: 0, 2004: 0, 2005: 0, 2006: 0, 2007: 0, 2008: 0, 2009: 0, 2010: 0, 2011: 0, 2012: 0, 2013: 1, 2014: 1, 2015: 0, 2016: 3, 2017: 2, 2018: 2, 2019: 1, 2020: 0, 2021: 0, 2022: 0}
save_sir_params 28 1.174697226916014 0.6324856833342344
70 {2000: 0, 2001: 0, 2002: 0, 2003: 0, 2004: 0, 2005: 0, 2006: 1, 2007: 1, 2008: 2, 2009: 4, 2010: 7, 2011: 11, 2012: 15, 2013: 24, 2014: 23, 2015: 58, 2016: 41, 2017: 44, 2018: 43, 2019: 44, 2020: 20, 2021: 12, 2022: 21}
save_sir_params 70 1.1348391401591982 0.5864903724072146
166 {2000: 0, 2001: 0, 2002: 0, 2003: 0, 2004: 0, 2005: 0, 2006: 0, 2007: 0, 2008: 0, 2009: 0, 2010: 0, 2011: 0, 2012: 0, 2013: 0, 2014: 0, 2015: 0, 2016: 0, 2017: 0, 2018: 0, 2019: 1, 2020: 0, 2021: 0, 2022: 0}
save_sir_params 166 0.81913045684113 0.3353959508063722
229 {2000: 0, 2001: 0, 2002: 0, 2003: 0, 2004: 0, 2005: 0, 2006: 0, 2007: 0, 2008: 0, 2009: 0, 2010: 0, 2011: 0, 2012: 0, 2013: 1, 2014: 2, 2015: 0, 2016: 2, 2017: 1, 2018: 1, 2019: 0, 

save_sir_params 6400 1.2800179458413934 0.6605461983567428
6500 {2000: 0, 2001: 0, 2002: 0, 2003: 0, 2004: 0, 2005: 0, 2006: 0, 2007: 0, 2008: 1, 2009: 1, 2010: 0, 2011: 0, 2012: 2, 2013: 9, 2014: 16, 2015: 20, 2016: 5, 2017: 3, 2018: 2, 2019: 2, 2020: 2, 2021: 0, 2022: 1}
save_sir_params 6500 1.3427597343933544 0.7227236455822259
6600 {2000: 0, 2001: 0, 2002: 0, 2003: 0, 2004: 0, 2005: 0, 2006: 0, 2007: 0, 2008: 1, 2009: 1, 2010: 2, 2011: 11, 2012: 5, 2013: 31, 2014: 38, 2015: 73, 2016: 54, 2017: 36, 2018: 12, 2019: 27, 2020: 11, 2021: 2, 2022: 11}
save_sir_params 6600 1.225549095167978 0.6335240522752543
6700 {2000: 0, 2001: 0, 2002: 0, 2003: 0, 2004: 0, 2005: 0, 2006: 0, 2007: 0, 2008: 0, 2009: 0, 2010: 0, 2011: 0, 2012: 16, 2013: 3, 2014: 1, 2015: 0, 2016: 1, 2017: 7, 2018: 3, 2019: 4, 2020: 3, 2021: 4, 2022: 2}
save_sir_params 6700 0.9625234117369873 0.0723541612982057
6800 {2000: 0, 2001: 0, 2002: 0, 2003: 0, 2004: 0, 2005: 0, 2006: 0, 2007: 0, 2008: 0, 2009: 0, 2010: 3, 2011: 0,

<Figure size 640x480 with 0 Axes>

In [5]:
import json
def predict_total_requests(city, year):
    with open("/Users/sararozin/Documents/Tama/Tama/total_request_per_year/models/sir/sir_model_v1.json") as sir_nodel_file:
        sir_models = json.load(sir_nodel_file)
        city_model = [model for model in sir_models if model['SemelIishuv'] == city][0]
        
    guess = infectedSIR(np.linspace(0, 30, 31), city_model['beta'], city_model['gamma'])
    guess /= np.sum(guess) 
    guess *= np.sum(x_data)
    return guess[year]

In [6]:
predict_total_requests(5000, 23)

3.514875747113986

In [7]:
import pandas as pd
def predict_total_requests_v2(city, year):
    out_df = pd.read_csv('/Users/sararozin/Documents/Tama/Tama/total_request_per_year/models/sir/out_v2.csv')
    return int(out_df[out_df['Yeshuv'] == city][str(year)])

In [8]:
predict_total_requests_v2(5000, 23)

109